In [47]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

In [48]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

In [49]:
x_val = x_train[50000:60000]
x_train = x_train[0:50000]
y_val = y_train[50000:60000]
y_train = y_train[0:50000]
print("train data has " + str(x_train.shape[0]) + " samples")
print("every train data is " + str(x_train.shape[1]) + "*" + str(x_train.shape[2]) + "image")

train data has 50000 samples
every train data is 28*28image


In [50]:
print("validation data has " + str(x_val.shape[0]) + " samples")
print("every train data is " + str(x_val.shape[1]) + "*" + str(x_val.shape[2]) + " images")

validation data has 10000 samples
every train data is 28*28 images


In [51]:
(x_train[0][8])

array([  0,   0,   0,   0,   0,   0,   0,  18, 219, 253, 253, 253, 253,
       253, 198, 182, 247, 241,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0], dtype=uint8)

In [52]:
import numpy as np
x_train = np.reshape(x_train, (50000,28,28,1))
x_val = np.reshape(x_val, (10000,28,28,1))
x_test = np.reshape(x_test, (10000,28,28,1))

print(x_train.shape)
print(x_test.shape)

(50000, 28, 28, 1)
(10000, 28, 28, 1)


In [53]:
# 데이터 정규화 (데이터의 모든 값이 0~255 범위 안에 있으므로, 255로 값을 나눠 0~1 사이로 정규화)
x_train = x_train.astype('float32')
x_val = x_val.astype('float32')
x_test = x_test.astype('float32')
gray_scale = 255
x_train /= gray_scale
x_val /= gray_scale
x_test /= gray_scale

In [54]:
# One-Hot 인코딩으로 변경 (손실함수에서 크로스 엔트로피를 계산하기 위함)
num_classes = 10
y_train = tf.keras.utils.to_categorical(y_train, num_classes)
y_val = tf.keras.utils.to_categorical(y_val, num_classes)
y_test = tf.keras.utils.to_categorical(y_test, num_classes)

In [55]:
x = tf.placeholder(tf.float32, shape=[None, 28, 28, 1])
y_ = tf.placeholder(tf.float32, shape=[None, 10])

In [56]:
def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)

def bias_variable(shape):
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)

In [57]:
def conv2d(x, W):
    return tf.nn.conv2d(x, W, strides=[1,1,1,1], padding='SAME')

def max_pool_2x2(x):
    return tf.nn.max_pool(x, ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME')

In [58]:
# 첫 번째 CONV 레이어는 총 16개의 필터를 가지고 있고, 각각 사이즈는 5 * 5
W_conv1 = weight_variable([5,5,1,16])
b_conv1 = bias_variable([16])  # 편향값은 필터 개수만큼 들어감

In [59]:
# 활성함수로 ReLU 사용
h_conv1 = tf.nn.relu(conv2d(x, W_conv1) + b_conv1)

In [60]:
# 풀링 레이어를 적용하여 맵의 크기 줄여줌
h_pool1 = max_pool_2x2(h_conv1)
# 28 * 28 에서 14 * 14 가 됨

In [61]:
# 두 번째 CONV 레이어는 총 32개의 필터를 가지고 있음
W_conv2 = weight_variable([5,5,16,32])
b_conv2 = bias_variable([32])
h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2)
h_pool2 = max_pool_2x2(h_conv2)
# 두 번째 풀링 레이어 이후 액티베이션 맵의 사이즈는 7 * 7 이 됨

In [62]:
# FC 를 통해 숫자를 분류하기 위해 모든 특징들을 하나의 배열로 형 변환
W_fc1 = weight_variable([7 * 7 * 32, 128])
b_fc1 = bias_variable([128])

h_pool2_flat = tf.reshape(h_pool2, [-1, 7*7*32])
h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)

In [63]:
# FC2 에서 10개의 노드를 가지고 소프트 맥스에 입력하여 각 노드별 확률을 구함
W_fc2 = weight_variable([128, 10])
b_fc2 = bias_variable([10])
y_conv = tf.matmul(h_fc1, W_fc2) + b_fc2

In [64]:
cross_entropy = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits_v2(
        labels=y_, logits=y_conv
    )
)

In [65]:
# 아담 옵티마이저를 통해 모델 최적화 (학습률 : 0.001 로 잡음)
train_step = tf.train.AdamOptimizer(0.001).minimize(cross_entropy)

In [66]:
correct_prediction = tf.equal(tf.argmax(y_conv,1), tf.argmax(y_, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [70]:
# 학습 및 테스트

# 초기화
init = tf.global_variables_initializer()

# 하이퍼 파라미터 세팅
epoch_cnt = 3
batch_size = 500
iteration = len(x_train) // batch_size

# 학습 시작
with tf.Session() as sess:
    tf.set_random_seed(777)
    # 세션 시작
    sess.run(init)
    for epoch in range(epoch_cnt):
        avg_loss = 0.
        start = 0; end = batch_size

        for i in range(iteration):
            if i%10 == 0:
                train_acc = accuracy.eval(
                    feed_dict={x:x_train[start: end],
                               y_: y_train[start: end]})
                print("step " + str(i) + ": train acc: " + str(train_acc))

            train_step.run(
            feed_dict={x:x_train[start: end], y_:y_train[start:end]})
        start += batch_size; end += batch_size

    # 모델 검증
    val_accuracy = accuracy.eval(feed_dict={x:x_val, y_: y_val})
    print("validation accuracy: "+ str(val_accuracy))

    test_accuracy = accuracy.eval(feed_dict={x:x_test, y_: y_test})
    print("test accuracy: " + str(test_accuracy))

step 0: train acc: 0.104
step 10: train acc: 0.576
step 20: train acc: 0.824
step 30: train acc: 0.878
step 40: train acc: 0.914
step 50: train acc: 0.944
step 60: train acc: 0.98
step 70: train acc: 0.996
step 80: train acc: 1.0
step 90: train acc: 1.0
step 0: train acc: 1.0
step 10: train acc: 1.0
step 20: train acc: 1.0
step 30: train acc: 1.0
step 40: train acc: 1.0
step 50: train acc: 1.0
step 60: train acc: 1.0
step 70: train acc: 1.0
step 80: train acc: 1.0
step 90: train acc: 1.0
step 0: train acc: 1.0
step 10: train acc: 1.0
step 20: train acc: 1.0
step 30: train acc: 1.0
step 40: train acc: 1.0
step 50: train acc: 1.0
step 60: train acc: 1.0
step 70: train acc: 1.0
step 80: train acc: 1.0
step 90: train acc: 1.0
validation accuracy: 0.9073
test accuracy: 0.9019
